# SoS Docker Guide

This documentation is under development.

## What is docker and why it is helpful

This is a big question to answer but in essence you can think docker containers as virtual machines with applications but without the bulky OS part, or applications with stripped down OSes. Docker contains are much more lightweight than virtual machines because all docker containers share the same core OS and related containers (e.g. different applications derived from the same CentOS or Ubuntu OS) share the same base container. Please refer to the [docker website](https://www.docker.com/) for details about docker. I have found it helpful to watch a few youtube videos on docker.

The reason why docker is very helpful in building (bioinformatics) pipelines are that 

1. Applications are encapsulated in docker containers so that they do not interfere with the underlying OS, and with other applications. For example, we can run a workflow with applications that based on different versions of Python2 and Python 3 without having to install them locally and calling the correct version of Python, because all applications use the specific version of Python and required libraries and tools inside their own container.

2. Workflows will be more stable and reproducible because unlike a local installation of Python that can be affected by other software and upgrades of python, Docker containers are stable and will not change.

3. The same docker containers can be executed on different OS (e.g. various version of Linux, MacOSX etc) so your workflow built on a Mac OS workstation can be executed on a cluster environment. 

There are of course some complexity in the use of docker but SoS has made it extremely easy to use docker in your workflows. 

## Installing and configuring docker

Docker is relatively new and is evolving very fast. It is crucial for you to install the latest version from [docker website](https://www.docker.com/). This website provides very detailed step by step instruction and you should have no problem installing docker on your machine. 

After installation, you should be able to start a docker terminal (On Mac, it is called *Docker QuickStart Terminal*) and run command

```bash
$ docker run hello-world
```

as suggested by the documentation. 

### Configuration for Mac (older version of Docker running on VirtualBox)

If you are using a Mac machine, docker containers are executed under a VirtualBox virtual machine. The docker daemon cannot see anything outside of `/Users` so if your data resides in a large volume outside of `/Users`, you will not be able to access it from inside a docker container. This can be very frustrating because Mac OS X's boot disk is usually very small so all important data likely reside in separate volumes under `/Volumes`.

In addition, the default docker machine has only 1 CPU, 2GB of RAM and 20GB diskspace. If you are using docker for any serious bioinformatics work and download a few docker images, you will need to add more RAM and more storage to the virtual box. To solve these problems, you would need to remove the default virtual machine and create a new one. 

From your command line, stop and remove the default virtual machine
```bash
$ docker-machine stop
$ docker-machine rm default
```
then create a new one with 
```bash
$ docker-machine create -d virtualbox --virtualbox-cpu-count 4 \
    --virtualbox-memory 30000 --virtualbox-disk-size 100000 default
```
Here I am using 4CPU/core, 30G of RAM and 100G of disk. You can adjust these parameters according to the configuration of your machine. 

After this, you would need to share `/volumes` to the virtual machine,

```bash
# stop docker virtual machine
$ docker-machine stop
# add all /Volumes to /Volumes
$ VBoxManage sharedfolder add default --name Volumes --hostpath /Volumes --automount
```

You would also need to mount `/Volumes` to the docker-machine using the following command after the docker machine starts,
```bash
$ docker-machine ssh default 'sudo mkdir -p /Volumes ; sudo  mount -t vboxsf Volumes /Volumes'
```
but SoS would do this automatically for you if you are using SoS to interact with docker.

## Running a workflow with docker

Running a docker-based workflow is easy because SoS will automatically download docker images and execute scripts inside docker container. Just make sure you start the script from a docker terminal (e.g. test with command `docker ps`)

## Building a docker image

Building a docker image is usually done outside of SoS if you are maintaining a collection of docker containers to be shared by your workflows, your groups, or everyone. However, if you need to create a docker image on-the-fly or would like to embed the Dockerfile inside a SoS script, you can use the `docker_build` action to build a docker container.

For example, you can build a container for MISO as follows:

```
[miso_build]
# building miso from a Dockerfile
docker_build: tag='mdabioinfo/miso:latest'

    ############################################################
    # Dockerfile to build MISO container images
    # Based on Anaconda python
    ############################################################

    # Set the base image to anaconda Python 2.7 (miso does not support python 3)
    FROM continuumio/anaconda

    # File Author / Maintainer
    MAINTAINER Bo Peng <bpeng@mdanderson.org>

    # Update the repository sources list
    RUN apt-get update

    # Install compiler and python stuff, samtools and git
    RUN apt-get install --yes \
     build-essential \
     gcc-multilib \
     gfortran \ 
     apt-utils \
     libblas3 \ 
     liblapack3 \
     libc6 \
     cython \ 
     samtools \
     libbam-dev \
     bedtools \
     wget \
     zlib1g-dev \ 
     tar \
     gzip

    WORKDIR /usr/local
    RUN pip install misopy
```

Command

```
sos run script miso_build
```

would build a docker image `mdabioinfo/miso:latest` that can be used by other SoS steps.

## Writing a workflow with docker support

Writing a workflow with docker support is a bit more complicated because you will need to understand a few concepts of docker, so reading through the [docker run manual](https://docs.docker.com/engine/reference/run/) should be helpful. The most important concept is **Volumes**, whch is how the host directories are mounted to a docker container so that the command executed inside the container can access (and change) files on the host machine. SoS helps the use of docker by

* Automatically mounts `/tmp` to `/tmp` 
* Automatically mounts `/Users` to `/Users` under MacOS X
* Automatically mounts user script inside docker and execute it as `/var/lib/sos/xxxxx`

so that step `input` and `output` are almost always identical inside and outside of docker. 

To use existing public docker container, you will need to specify its tag using option `docker_image`. For example, to use `compbio/ngseasy-fastqc` container to run `fastqc`, instead of installing `fastqc` locally, you can do

```
[MISO_1]
run:     docker_image='compbio/ngseasy-fastqc:1.0-r001'
    fastqc ${input} -o /tmp
```

(More to follow)

## Limitations

* Virtual Box virtual machine does not support symbolic link so running `ln -s` inside a docker machine under Mac will cause a strange error message `Read-only file system`.